## Pretrained Crafter Gameplay Demos

This demo notebook loads pretrained weights for a **pixel-based** Crafter agent and replays gameplay videos for user-specified instructions.

In [1]:
# Specify GPU device index for policy inference
GPU = 0

In [4]:
import random
import os
import glob
import warnings

import requests
from IPython.display import Video, HTML
import torch

popgym_example = __import__("02_popgym_suite")
import amago
from amago.cli_utils import *
from amago.envs.builtin.gym_envs import POPGymEnv

#### Step 1: Initialize a new agent with the correct architecture

In [8]:
# boilerplate to imitate training script command line interface
parser = ArgumentParser()
add_common_cli(parser)
args = parser.parse_args(["--run_name=amago_test", "--buffer_dir=checkpoints"])
args.no_log = True
args.gpu = GPU

# print(args)

config = {
    "amago.agent.Agent.reward_multiplier" : 10.,
    "amago.agent.Agent.tstep_encoder_Cls" : partial(popgym_example.switch_tstep_encoder, obs_kind="crop"),
    "amago.nets.tstep_encoders.TstepEncoder.goal_emb_Cls" : amago.nets.goal_embedders.TokenGoalEmb,
    "amago.nets.goal_embedders.TokenGoalEmb.zero_embedding" : False,
    "amago.nets.goal_embedders.TokenGoalEmb.goal_emb_dim" : 64,
}

switch_traj_encoder(config, arch="transformer", memory_size=256, layers=3)
use_config(config, args.configs, finalize=False)

make_env = lambda: POPGymEnv(f"popgym-{'AutoencodeMedium'}-v0")

group_name = f"popgym"
run_name = group_name + "_trial_0"
experiment = create_experiment_from_cli(
            args,
            make_train_env=make_env,
            make_val_env=make_env,
            max_seq_len=512,
            traj_save_len=2501,
            stagger_traj_file_lengths=False,
            run_name=run_name,
            group_name=group_name,
            # batch_size=24, # 18
            val_timesteps_per_epoch=5000,
            relabel="some",
            goal_importance_sampling=True,
        )

# build the agent with randomly initialized weights. This is a smaller architecture than used in the main results
experiment.start()

TypeError: switch_tstep_encoder() missing 2 required positional arguments: 'config' and 'arch'
  In call to configurable 'Agent' (<class 'amago.agent.Agent'>)

#### Step 2: Download and replace parameters with pretrained checkpoint

In [4]:
# checkpoint from long pixel-based training run that closely reproduces Appendix C5 Table 2 using the public repo
ckpt_link = "https://utexas.box.com/shared/static/xvkgo02vkp8kn7j80051jbr6224tep9r.pt"
response = requests.get(ckpt_link)

# write file to the expected name and location based on this experiment's configuration (above).
with open(os.path.join("crafter_pretrained_example", run_name, "ckpts", "crafter_dec23_directed_crafter_crop_trial_0_BEST.pt"), "wb") as f:
    f.write(response.content)

# load checkpoint
# you would normally load the best checkpoint like this:
# experiment.load_checkpoint(loading_best=True)
# manual workaround for backwards-compatible old checkpoint
ckpt = torch.load(os.path.join(experiment.ckpt_dir, f"{experiment.run_name}_BEST.pt"), map_location=experiment.DEVICE)
experiment.policy.load_state_dict(ckpt["model_state"], strict=False)

_IncompatibleKeys(missing_keys=['maximized_critics.inp_layer.weight', 'maximized_critics.inp_layer.bias', 'maximized_critics.core_layers.0.weight', 'maximized_critics.core_layers.0.bias', 'maximized_critics.output_layer.weight', 'maximized_critics.output_layer.bias'], unexpected_keys=[])

#### Step 3: Evaluate and visualize

In [5]:
# Specify the task here! A list of any of up to 5 of the crafter achievements with _ separators. For example:
TASK = ["make_stone_pickaxe", "collect_coal", "travel_40m_40m", "place_stone"]

: 

In [6]:
# reset video directory for a new task
files = glob.glob('crafter_notebook_videos/*')
for f in files:
    os.remove(f)

def make_eval_env():
    e = make_env()
    e.set_env_name(f"crafter_eval")
    # manually set the task
    e.set_fixed_task([t.split("_") for t in TASK])
    return e

experiment.parallel_actors = 1 # 6 # adjust as needed!

# runs the evaluation and saves videos to disk
warnings.filterwarnings("ignore", category=UserWarning)
success = experiment.evaluate_test(make_eval_env, timesteps=3500, render=False)["Average Success Rate in crafter_eval"]
print(f"\n\nTask \"{', '.join(TASK)}\" Success Rate: {success * 100 : .1f}%")

Env Interaction:   0%|          | 0/3500 [00:00<?, ?it/s]/home/jovyan/.local/lib/python3.10/site-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


In [ ]:
# display gameplay videos in the notebook; run again for a new random sample
MAX_VIDEOS = 8
videos = glob.glob("crafter_notebook_videos/*")
random.shuffle(videos)
html_str = f"<table><caption style='font-size: 24px'>{', '.join(TASK)}</caption><tr>"
for i, video_path in enumerate(videos):
    video_html = f"""
    <td>
        <video width=300px alt="Video" controls>
            <source src="{video_path}" type="video/mp4">
        </video>
    </td>
    """
    html_str += video_html
    if (i + 1) % 4 == 0:
        html_str += "</tr><tr>"
    if i >= MAX_VIDEOS - 1:
        break
html_str += "</tr></table>"
HTML(html_str)

,,,
,,,
